Problem (Re)Statement:

* Shortness of breath (dyspnea) may be due to tuberculosis, lung cancer or bronchitis, or none of them, or more than one of them. 
* A recent visit to Asia increases the chances of tuberculosis.
* Smoking is known to be a risk factor for both lung cancer and bronchitis. 
* A positive chest X-ray suggests either lung cancer or tuberculosis, but cannot distinguish between them

Here is a data set to pull your model parameters from

In [1]:
from pandas import *
df = pandas.read_csv("asia.csv")

In [12]:
df.head()

<image src="asia.gif" size=200/>

<image src="asia.gif"/>


Begin by writing out your model.  For example here are names of some nodes, and the arcs that connect them.
<pre>
Asia                 -> Tuberculosis

Tuberculosis, Cancer -> TuberculosisOrCancer

TuberculosisOrCancer -> Dyspnea

</pre>

-- Informally Write Your Model In This Cell -- 
It will determine the parameters you will need to get from the data set
<pre>
Asia                  -> Tuberculosis

Tuberculosis          -> TuberculosisOrCancer

Cancer                -> TuberculosisOrCancer

TuberculossisOrCancer -> Xray

TuberculossisOrCancer -> Dyspnea

Smoking               -> Cancer

Smoking               -> Bronchitis

Bronchitis            -> Dyspnea
</pre>


Now define your distributions

In [3]:
from pomegranate import *
entries = len(df)

#P(smoking)
smoking = DiscreteDistribution({0: len(df[df.Smoker == 0]) / entries,
                                1: len(df[df.Smoker == 1]) / entries})

#P(Asia)
asia = DiscreteDistribution({0: len(df[df.VisitToAsia == 0]) / entries,
                             1: len(df[df.VisitToAsia == 1]) / entries})
                                
                            
#P(tb|Asia)
asia_0 = len(df[(df.VisitToAsia == 0)])
asia_1 = len(df[(df.VisitToAsia == 1)])
tb = ConditionalProbabilityTable([[0, 0, len(df[(df.VisitToAsia == 0) & (df.Tuberculosis == 0)])/asia_0],
                                  [0, 1, len(df[(df.VisitToAsia == 0) & (df.Tuberculosis == 1)])/asia_0],
                                  [1, 0, len(df[(df.VisitToAsia == 1) & (df.Tuberculosis == 0)])/asia_1],
                                  [1, 1, len(df[(df.VisitToAsia == 1) & (df.Tuberculosis == 1)])/asia_1]],[asia])
                             
#P(cancer|Smoking)
smoking_0 = len(df[(df.Smoker == 0)])
smoking_1 = len(df[(df.Smoker == 1)])
cancer= ConditionalProbabilityTable([[0, 0, len(df[(df.Smoker == 0) & (df.LungCancer == 0)])/smoking_0],
                                     [0, 1, len(df[(df.Smoker == 0) & (df.LungCancer == 1)])/smoking_0],
                                     [1, 0, len(df[(df.Smoker == 1) & (df.LungCancer == 0)])/smoking_1],
                                     [1, 1, len(df[(df.Smoker == 1) & (df.LungCancer == 1)])/smoking_1]],[smoking])

#P(bronchitis|Smoking)
bronc= ConditionalProbabilityTable([[0, 0, len(df[(df.Smoker == 0) & (df.Bronchitis == 0)])/smoking_0],
                                    [0, 1, len(df[(df.Smoker == 0) & (df.Bronchitis == 1)])/smoking_0],
                                    [1, 0, len(df[(df.Smoker == 1) & (df.Bronchitis == 0)])/smoking_1],
                                    [1, 1, len(df[(df.Smoker == 1) & (df.Bronchitis == 1)])/smoking_1]],[smoking])


#P(tb_Cancer|Tb,Cancer)
tbcancer_00 = len(df[(df.Tuberculosis == 0) & (df.LungCancer == 0)])
tbcancer_01 = len(df[(df.Tuberculosis == 0) & (df.LungCancer == 1)])
tbcancer_10 = len(df[(df.Tuberculosis == 1) & (df.LungCancer == 0)])
tbcancer_11 = len(df[(df.Tuberculosis == 1) & (df.LungCancer == 1)])
tb_cancer = ConditionalProbabilityTable([[0,0,0, len(df[(df.Tuberculosis == 0) & (df.LungCancer == 0) & (df.TuberculosisOrCancer == 0)])/tbcancer_00],
                                         [0,0,1, len(df[(df.Tuberculosis == 0) & (df.LungCancer == 0) & (df.TuberculosisOrCancer == 1)])/tbcancer_00],
                                         [0,1,0, len(df[(df.Tuberculosis == 0) & (df.LungCancer == 1) & (df.TuberculosisOrCancer == 0)])/tbcancer_01],
                                         [0,1,1, len(df[(df.Tuberculosis == 0) & (df.LungCancer == 1) & (df.TuberculosisOrCancer == 1)])/tbcancer_01],
                                         [1,0,0, len(df[(df.Tuberculosis == 1) & (df.LungCancer == 0) & (df.TuberculosisOrCancer == 0)])/tbcancer_10],
                                         [1,0,1, len(df[(df.Tuberculosis == 1) & (df.LungCancer == 0) & (df.TuberculosisOrCancer == 1)])/tbcancer_10],
                                         [1,1,0, len(df[(df.Tuberculosis == 1) & (df.LungCancer == 1) & (df.TuberculosisOrCancer == 0)])/tbcancer_11],
                                         [1,1,1, len(df[(df.Tuberculosis == 1) & (df.LungCancer == 1) & (df.TuberculosisOrCancer == 1)])/tbcancer_11]], 
                                         [tb, cancer])

#P(XRay|tb_Cancer)
tb_Cancer_0 = len(df[(df.TuberculosisOrCancer == 0)])
tb_Cancer_1 = len(df[(df.TuberculosisOrCancer == 1)])
xray = ConditionalProbabilityTable([[0, 0, len(df[(df.TuberculosisOrCancer == 0) & (df.XRay == 0)])/tb_Cancer_0],
                                    [0, 1, len(df[(df.TuberculosisOrCancer == 0) & (df.XRay == 1)])/tb_Cancer_0],
                                    [1, 0, len(df[(df.TuberculosisOrCancer == 1) & (df.XRay == 0)])/tb_Cancer_1],
                                    [1, 1, len(df[(df.TuberculosisOrCancer == 1) & (df.XRay == 1)])/tb_Cancer_1]],
                                    [tb_cancer])

#P(Dyspnea|tb_Cancer, bronc)
tbcbronc_00 = len(df[(df.TuberculosisOrCancer == 0) & (df.Bronchitis == 0)])
tbcbronc_01 = len(df[(df.TuberculosisOrCancer == 0) & (df.Bronchitis == 1)])
tbcbronc_10 = len(df[(df.TuberculosisOrCancer == 1) & (df.Bronchitis == 0)])
tbcbronc_11 = len(df[(df.TuberculosisOrCancer == 1) & (df.Bronchitis == 1)])
dysp = ConditionalProbabilityTable([[0,0,0, len(df[(df.TuberculosisOrCancer == 0) & (df.Bronchitis == 0) & (df.Dyspnea == 0)])/tbcbronc_00],
                                    [0,0,1, len(df[(df.TuberculosisOrCancer == 0) & (df.Bronchitis == 0) & (df.Dyspnea == 1)])/tbcbronc_00],
                                    [0,1,0, len(df[(df.TuberculosisOrCancer == 0) & (df.Bronchitis == 1) & (df.Dyspnea == 0)])/tbcbronc_01],
                                    [0,1,1, len(df[(df.TuberculosisOrCancer == 0) & (df.Bronchitis == 1) & (df.Dyspnea == 1)])/tbcbronc_01],
                                    [1,0,0, len(df[(df.TuberculosisOrCancer == 1) & (df.Bronchitis == 0) & (df.Dyspnea == 0)])/tbcbronc_10],
                                    [1,0,1, len(df[(df.TuberculosisOrCancer == 1) & (df.Bronchitis == 0) & (df.Dyspnea == 1)])/tbcbronc_10],
                                    [1,1,0, len(df[(df.TuberculosisOrCancer == 1) & (df.Bronchitis == 1) & (df.Dyspnea == 0)])/tbcbronc_11],
                                    [1,1,1, len(df[(df.TuberculosisOrCancer == 1) & (df.Bronchitis == 1) & (df.Dyspnea == 1)])/tbcbronc_11]], 
                                    [tb_cancer, bronc])

Next define the nodes in your network

In [4]:
asia_node = Node(asia, name="asia")
smoking_node = Node(smoking, name="smoking")
tb_node = Node(tb, name="tb")
cancer_node = Node(cancer, name="cancer")
bronc_node = Node(bronc, name="bronc")
tb_cancer_node = Node(tb_cancer, name="tb_cancer")
xray_node = Node(xray, name="xray")
dysp_node = Node(dysp, name="dysp")

Define your model, adding states and edges

In [5]:
model = BayesianNetwork("Bayesian Network Example")

model.add_states(asia_node, smoking_node, tb_node, cancer_node, bronc_node, tb_cancer_node, xray_node, dysp_node)

model.add_edge(asia_node, tb_node)
model.add_edge(tb_node, tb_cancer_node)
model.add_edge(cancer_node, tb_cancer_node)
model.add_edge(tb_cancer_node, xray_node)
model.add_edge(tb_cancer_node, dysp_node)
model.add_edge(smoking_node, cancer_node)
model.add_edge(smoking_node, bronc_node)
model.add_edge(bronc_node, dysp_node)

model.bake()

------------------------------------------------

#### Questions

1.  Before checking, write down your guess for the probability that an individual in the population at large would have a positive X-Ray (i.e. a result that suggests either lung cancer or tuberculosis)

Guesstimate of positive X-Ray in the population: 0.0488 or 4.88%

2.  Now read that probability from the model you built.   

In [11]:
model.predict_proba({})[6].parameters[0][1]

3.  Is your empirical result significantly different from your guess?  If so, explain why.  If not, explain how you came to your original guess.

My original guess of the population getting a positive on XRay is close to the actual result but it's still off about 6 percent. 

We didn't use the asia.csv to filter on X-Ray to formulate my guess. To calculate my guess, we noticed X-Ray node is derived from multiple nodes(tuberculosis, lung cancer, asia, and smoking). For a high percentage of positive X-Ray, percentage of it's parents and grandparents nodes have to be high. However, it's very unlikely for the population to have high percentages of tuberculosis, asia, lung cancer, and smoking at the same time.

So I took the probability of positive X-Ray given Tuberculosis and Cancer to get:
0.050272756444539525 x 0.9723502304147466 = ~4.88%

4.  How much does a trip to Asia affect the likelihood of an individual having Dyspnea?

In [10]:
model.predict_proba({"asia": 1})[7].parameters[0][1]

 -- 45.39% likelihood of having Dyspnea after a trip to Asia -- 

5.  Suppose you are a nonsmoker individual presenting with Dyspnea and you have never been to Asia.  In your panic you have a chest XRay done, which comes out negative.   What do you now know about the state of your health?

In [9]:
#predict having Tuberculosis or Lung Caner
print("Having Tuberculosis or Lung Cancer ", 
      model.predict_proba({"smoking": 0, "dysp": 1, "asia": 0, "xray": 0})[5].parameters)

#predict having Bronchitis
print("Having Bronchitis ", 
    model.predict_proba({"smoking": 0, "dysp": 1, "asia": 0, "xray": 0})[4].parameters)

-- 99.88% not having Tuberculosis or Lung Cancer but 78.00% having Bronchitis -- 